# Start Date

In [2]:
# input
# period = 1
# start_date = '2023-07-01'

# 1. Import Packages and Data

In [3]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
import datetime as dt

In [4]:
#manual input
df = pd.read_csv('SS_LC_merged_data.csv')

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,25,26,27,28,29,30,31,32,33,34,35,36,43,47,50,53,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# SMS testing pipeline
sms_test = infrastructure.get_smartsheet('test_sms')
sms_test['Hitpath ID'] = sms_test['Hitpath ID'].astype(str)
# SMS offer sheet 
sms_offer = infrastructure.get_smartsheet('offers_sms')
sms_offer = sms_offer[sms_offer['Hitpath ID'].isna() == False]
sms_offer['Hitpath ID'] = sms_offer['Hitpath ID'].astype(int).astype(str)
# Email offer sheet 
email_offer = infrastructure.get_smartsheet('offers_email')
# get mamba for shortcode 
mamba = infrastructure.get_mamba()
# Offer Testing Timeline 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
timeline = gc.open_by_url('https://docs.google.com/spreadsheets/d/16vrHMWs0ambcBJ1sC0SqpYVG8SOSWlbg1N22-bF49v8/edit#gid=423634805') 
timeline_wks  = timeline[1].get_as_df()

In [6]:
#la nina
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_wks  = lanina[1].get_as_df()

# 2. Segment Size

In [7]:
#input
offer_id_test = '12764'
test_offer_info = sms_test.loc[sms_test['Hitpath ID'] == offer_id_test]
payout = float(test_offer_info['$ Payout'].replace(r'[^0-9\.]','',regex=True))
if(payout  > 50.00):
    payout = payout/2
# payout = 8.00
test_offer_vertical = test_offer_info.loc[test_offer_info['Hitpath ID'] == offer_id_test, 'Vertical'].iloc[0]
expected_revenue = payout*1.6+20

In [44]:
#last 60 days of data
last60days= pd.Timestamp(dt.date.today()-dt.timedelta(days=60)).strftime('%Y-%m-%d')
df = df[df['Date']>=last60days]
df['Hitpath_Offer_ID'] = df['Hitpath_Offer_ID'].astype(str)
df['Hitpath_Offer_ID'] = df['Hitpath_Offer_ID'].replace(r'\.0$', '', regex=True)
df = df[df['Send Strategy'] != 'AR']



In [45]:
#median delivered volume by shortcode_DP.SV
median_delivered = df.groupby(['Date','shortcode_DP.SV'])['Delivered'].sum().reset_index()
median_delivered = median_delivered.groupby(['shortcode_DP.SV'])['Delivered'].median().reset_index()

In [46]:
#eCPM by shortcode_DP.SV
eCPM_df = df.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Delivered':'sum'})
eCPM_df['eCPM'] = eCPM_df['Revenue'] * 1000 / eCPM_df['Delivered']

In [47]:
#merge eCPM and median deliverd
merged_segment_size = median_delivered.merge(eCPM_df,how = 'right', on = 'shortcode_DP.SV')
merged_segment_size = merged_segment_size.rename(columns=
                                                 {"Delivered_x": "Median Delivered", 
                                                  "Delivered_y": "Total Delivered"})

In [48]:
merged_segment_size['segment_size'] = expected_revenue * 1000 / merged_segment_size['eCPM']
merged_segment_size['segment_size'] -= merged_segment_size['segment_size'] % -100

In [49]:
merged_segment_size

,shortcode_DP.SV,Median Delivered,Revenue,Total Delivered,eCPM,segment_size
0,DSS_RHD.CC,1529.0,1063.2300,86708.0,12.262190,5900.0
1,DSS_TLG.PL,9860.0,8740.9800,480361.0,18.196690,4000.0
2,DSS_ZM.PL.2,4755.0,6527.4200,276578.0,23.600648,3100.0
3,FLC_AI.CC,3194.0,3851.0800,194346.0,19.815587,3600.0
4,FLC_EDM.247L,13744.0,21036.7855,789043.0,26.661140,2700.0
5,FLC_I.CC,1081.0,1230.7600,57423.0,21.433224,3400.0
6,FLC_I.RC,1127.0,1537.6100,75385.0,20.396763,3500.0
7,FLC_NPD.RTO,969.0,2518.8200,74424.0,33.844190,2200.0
8,FLC_PN.FC,5696.0,9315.1900,336180.0,27.708936,2600.0
9,FLC_PN.SWP,34656.0,36603.9355,2012290.0,18.190189,4000.0


# 3. Offer Vertical

In [50]:
#merge vertical with offer per drop
sms_offer = sms_offer.rename(columns={"Hitpath ID": "Hitpath_Offer_ID"})
vertical_merge = df.merge(sms_offer[['Hitpath_Offer_ID','Vertical']],on='Hitpath_Offer_ID', how = 'left')

In [51]:
#vertical eCPM by shortcode_dp.sv
vertical_eCPM_DF = vertical_merge.groupby(['shortcode_DP.SV','Vertical']).agg(
    {'Revenue':'sum','Delivered':'sum'}).reset_index()
vertical_eCPM_DF['Offer Vertical eCPM'] = vertical_eCPM_DF['Revenue'] * 1000 / vertical_eCPM_DF['Delivered']

In [52]:
# vertical_eCPM_DF.to_clipboard()

# 4. Testing

In [53]:
#manual vertical if not enough data/payout
# test_offer_vertical = 'Education'

In [62]:
#similar buckets
bucket1 = ['Loan', 'Debt', 'Credit Score', 
           'Credit Card', 'Financial', 'Shopping', 'Business Opportunity', 'Bank Account', 'Tax Debt']
bucket2 = ['Cellular', 'App Install', 'Sweepstakes', 'Gift Card', 'Gas Card', 'Freebies', 'Surveys', 'Promos']
bucket3 = ['Education', 'Benefits','Resources', 'Rent To Own', 'Employment','Delivery', 'Miscellaneous', 
           'Tort', 'Legal', 'Injury Claim', 'Identity Protection']
bucket4 = ['Auto Warranty', 'Auto Loan', 'Auto Insurance', 'Auto Purchase', 
           'Home Security', 'Health', 'Healthcare', 'Home Loan', 'Home Warranty', 'Home Refinance']



In [63]:
buckets = pd.DataFrame({'bucket1': pd.Series(bucket1), 'bucket2': pd.Series(bucket2), 'bucket3': pd.Series(bucket3),
             'bucket4': pd.Series(bucket4)})

In [64]:
vertical_eCPM = vertical_merge.groupby('Vertical').agg({'Revenue':'sum','Delivered':'sum'}).reset_index()
vertical_eCPM['Vertical eCPM'] = vertical_eCPM['Revenue'] * 1000 / vertical_eCPM['Delivered']
vertical_eCPM

,Vertical,Revenue,Delivered,Vertical eCPM
0,App Install,0.000,4474.0,0.000000
1,Auto Insurance,20.000,4197.0,4.765309
2,Auto Warranty,12108.000,512471.0,23.626703
3,Bank Account,0.000,5811.0,0.000000
4,Cellular,294.000,27383.0,10.736588
5,Coupons,43.050,5169.0,8.328497
6,Credit Card,31810.500,2164114.0,14.699087
7,Credit Score,1377.600,106658.0,12.916049
8,Cyber Security,0.000,11895.0,0.000000
9,Debt,29.400,5659.0,5.195264


In [65]:
# 

In [66]:
if 'test' in locals():
    del test
if 'test2' in locals():
    del test2
if((vertical_eCPM.loc[vertical_eCPM['Vertical'] == test_offer_vertical, 'Delivered'].size == 1) == True):
    if(vertical_eCPM.loc[vertical_eCPM['Vertical'] == test_offer_vertical, 'Delivered'].iloc[0]) < 100000:
        for column in buckets:
            if(buckets[column].eq(test_offer_vertical)).any():
                sim = buckets[column]
                bucket = vertical_eCPM_DF.loc[vertical_eCPM_DF['Vertical'].isin(sim)]
                bucket_merged = bucket.merge(merged_segment_size[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.merge(median_delivered[['shortcode_DP.SV','Delivered']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.rename(columns={"Delivered_x": "Total Delivered", "Delivered_y": "Median Delivered"})
                bucket_merged['% Testing Size'] = bucket_merged['segment_size']/bucket_merged['Median Delivered']
                test = bucket_merged.loc[((bucket_merged['Median Delivered'] < 10000) & 
                          (bucket_merged['Median Delivered'] >= 4000) & 
                          (bucket_merged['% Testing Size'] <= .35)) |
                          ((bucket_merged['Median Delivered'] >= 10000) &
                          (bucket_merged['% Testing Size'] <= .2))
                         ].sort_values(by = "Offer Vertical eCPM",ascending = False)
                test2 = test.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Total Delivered':'sum'}).reset_index()
                test2['Bucket eCPM'] = test2['Revenue'] * 1000 / test2['Total Delivered']
                test2 = test2.merge(merged_segment_size[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                test2 = test2.merge(median_delivered[['shortcode_DP.SV','Delivered']],on='shortcode_DP.SV', how = 'left')
                test2['% Testing Size'] = test2['segment_size']/test2['Delivered']
                test2

                
            else:
                print("Bucket not found in " + column)
    else:
        bucket = vertical_eCPM_DF.loc[vertical_eCPM_DF['Vertical'] == test_offer_vertical]
        bucket_merged = bucket.merge(merged_segment_size[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
        bucket_merged = bucket_merged.merge(median_delivered[['shortcode_DP.SV','Delivered']],on='shortcode_DP.SV', how = 'left')
        bucket_merged = bucket_merged.rename(columns={"Delivered_x": "Total Delivered", "Delivered_y": "Median Delivered"})
        bucket_merged['% Testing Size'] = bucket_merged['segment_size']/bucket_merged['Median Delivered']
        test = bucket_merged.loc[((bucket_merged['Median Delivered'] < 10000) & 
                      (bucket_merged['Median Delivered'] >= 4000) & 
                      (bucket_merged['% Testing Size'] <= .35)) |
                      ((bucket_merged['Median Delivered'] >= 10000) &
                      (bucket_merged['% Testing Size'] <= .2))
                     ].sort_values(by = "Offer Vertical eCPM",ascending = False)
else: 
    for column in buckets:
            if(buckets[column].eq(test_offer_vertical)).any():
                sim = buckets[column]
                bucket = vertical_eCPM_DF.loc[vertical_eCPM_DF['Vertical'].isin(sim)]
                bucket_merged = bucket.merge(merged_segment_size[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.merge(median_delivered[['shortcode_DP.SV','Delivered']],on='shortcode_DP.SV', how = 'left')
                bucket_merged = bucket_merged.rename(columns={"Delivered_x": "Total Delivered", "Delivered_y": "Median Delivered"})
                bucket_merged['% Testing Size'] = bucket_merged['segment_size']/bucket_merged['Median Delivered']
                test = bucket_merged.loc[((bucket_merged['Median Delivered'] < 10000) & 
                          (bucket_merged['Median Delivered'] >= 4000) & 
                          (bucket_merged['% Testing Size'] <= .35)) |
                          ((bucket_merged['Median Delivered'] >= 10000) &
                          (bucket_merged['% Testing Size'] <= .2))
                         ].sort_values(by = "Offer Vertical eCPM",ascending = False)
                test2 = test.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Total Delivered':'sum'}).reset_index()
                test2['Bucket eCPM'] = test2['Revenue'] * 1000 / test2['Total Delivered']
                test2 = test2.merge(merged_segment_size[['shortcode_DP.SV','segment_size']],on='shortcode_DP.SV', how = 'left')
                test2 = test2.merge(median_delivered[['shortcode_DP.SV','Delivered']],on='shortcode_DP.SV', how = 'left')
                test2['% Testing Size'] = test2['segment_size']/test2['Delivered']
                test2
            else:
                print("Bucket not found in " + column)
        

Bucket not found in bucket2
Bucket not found in bucket3
Bucket not found in bucket4


In [67]:
if 'test2' in locals():
    print(test2)
else:
    print(test)

  shortcode_DP.SV     Revenue  Total Delivered  Bucket eCPM  segment_size  \
0    FLC_EDM.247L  20973.8355         777753.0    26.967219        2700.0   
1      FLC_PN.SWP   9836.8355         701929.0    14.014004        4000.0   
2      MBC_PN.SWP   3647.2800         434698.0     8.390377        4600.0   

   Delivered  % Testing Size  
0    13744.0        0.196449  
1    34656.0        0.115420  
2    31223.0        0.147327  


In [68]:
test2.sort_values(by = "Bucket eCPM",ascending = False)

,shortcode_DP.SV,Revenue,Total Delivered,Bucket eCPM,segment_size,Delivered,% Testing Size
0,FLC_EDM.247L,20973.8355,777753.0,26.967219,2700.0,13744.0,0.196449
1,FLC_PN.SWP,9836.8355,701929.0,14.014004,4000.0,34656.0,0.115420
2,MBC_PN.SWP,3647.2800,434698.0,8.390377,4600.0,31223.0,0.147327


In [69]:
test

,shortcode_DP.SV,Vertical,Revenue,Total Delivered,Offer Vertical eCPM,segment_size,Median Delivered,% Testing Size
10,FLC_EDM.247L,Loan,17207.5855,547163.0,31.448737,2700.0,13744.0,0.196449
22,FLC_PN.SWP,Loan,4160.7355,222790.0,18.675594,4000.0,34656.0,0.115420
8,FLC_EDM.247L,Credit Card,3761.0000,228595.0,16.452678,2700.0,13744.0,0.196449
21,FLC_PN.SWP,Credit Score,1377.6000,106658.0,12.916049,4000.0,34656.0,0.115420
20,FLC_PN.SWP,Credit Card,4298.5000,370015.0,11.617097,4000.0,34656.0,0.115420
71,MBC_PN.SWP,Credit Card,3074.5000,346832.0,8.864522,4600.0,31223.0,0.147327
72,MBC_PN.SWP,Loan,572.7800,84521.0,6.776777,4600.0,31223.0,0.147327
9,FLC_EDM.247L,Debt,5.2500,1995.0,2.631579,2700.0,13744.0,0.196449
19,FLC_PN.SWP,Bank Account,0.0000,2466.0,0.000000,4000.0,34656.0,0.115420
70,MBC_PN.SWP,Bank Account,0.0000,3345.0,0.000000,4600.0,31223.0,0.147327
